In [ ]:
!pip install nltk


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [ ]:
# Download NLTK data (stopwords and tokenizer)
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Dataset/Twitter_Data.csv')

In [ ]:
import numpy as np

# Remove missing values (NaN)
data = data[data['clean_text'].notna()]

# Convert non-string values to strings
data['clean_text'] = data['clean_text'].apply(lambda x: str(x))

In [ ]:
# Assuming you have a DataFrame named 'data'
import pandas as pd

# Remove rows with missing values from the entire DataFrame
data.dropna(inplace=True)

# To remove missing values from a specific column, for example 'label'
data.dropna(subset=['clean_text'], inplace=True)


In [ ]:
# Assuming your dataset has 'text' and 'label' columns
text_data = data['clean_text']
labels = data['category']

In [ ]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and punctuation
    text = ''.join([char for char in text if char not in string.punctuation])

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]

    # Join the tokens back into a string
    return ' '.join(tokens)

# Apply the preprocessing function to the text data
text_data = text_data.apply(preprocess_text)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Create a TF-IDF vectorizer with a maximum of 5000 features
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the preprocessed text data to create the TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(text_data)


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, labels, test_size=0.2, random_state=42)

In [ ]:
# Create an SVM classifier (you can tune hyperparameters)
classifier = SVC(kernel='linear')
classifier.fit(X_train, y_train)


In [ ]:
# Make predictions on the test set
predictions = classifier.predict(X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

In [ ]:
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.9028655580781739
Classification Report:
               precision    recall  f1-score   support

        -1.0       0.88      0.82      0.85      7152
         0.0       0.88      0.98      0.93     11067
         1.0       0.93      0.89      0.91     14375

    accuracy                           0.90     32594
   macro avg       0.90      0.89      0.89     32594
weighted avg       0.90      0.90      0.90     32594



In [ ]:
# New tweets you want to classify
new_tweets = ["Just had the worst customer service experience at [Company Name]. They need to work on their support.", "answer who among these the most powerful world leader today trump putin modi may ", "through our vote ensure govt need and deserve anupam kher responds modis appeal for the 2019 elections "]

In [ ]:
# Preprocess and TF-IDF vectorize the new tweets
new_tweets = [preprocess_text(tweet) for tweet in new_tweets]
new_tweet_features = tfidf_vectorizer.transform(new_tweets)

In [ ]:
# Make predictions
predictions = classifier.predict(new_tweet_features)

In [ ]:
for tweet, prediction in zip(new_tweets, predictions):
    if prediction == -1:
        label = "Hate"
    elif prediction == 0:
        label = "Neutral"
    else:
        label = "Non-Hate"
    print(f'Tweet: "{tweet}", Prediction: {label}')

Tweet: "worst customer service experience company name need work support", Prediction: Hate
Tweet: "answer among powerful world leader today trump putin modi may", Prediction: Non-Hate
Tweet: "vote ensure govt need deserve anupam kher responds modis appeal 2019 elections", Prediction: Neutral
